# TypeDispatch POC
> Displaying typedispatch in show_doc

In [1]:
#|default_exp dispatch

In [2]:
#|export
import inspect
from inspect import getdoc
from operator import is_not
from functools import partial

import fastcore
from fastcore.basics import *
from fastcore.foundation import *
from fastcore.dispatch import typedispatch
from fastcore.docments import _docments, docments


from nbprocess.showdoc import _wrap_sig, _fmt_sig, _signature
from nbprocess.showdoc import *
from nbprocess.read import get_config 

In [3]:
from fastcore.test import test_eq

In [4]:
@typedispatch
def type_sum(
    a:int, # The first docment
    b:int # The second docment
) -> int: 
    "Returns sum of `a` and `b`"
    return a+b

@typedispatch
def type_sum(
    a:float,
    b:int,
) -> float: 
    return a+b

@typedispatch
def type_sum(
    a:float,
    b:float
) -> float:
    return a+b

In [5]:
#export
@patch
def concat(self:L):
    "Concatenates `self`"
    return L(concat(self))

In [6]:
o = L([[1,2], [3]])
test_eq(o.concat(), L([1,2,3]))

In [7]:
#export
def trace_dispatch(func):
    "Returns a list of all signatures from a type dispatch"
    return L(type_dict.keys()).map(
        lambda x: [
            inspect.signature(o) 
            for o in type_dict[x].d.values()
        ]
    ).concat()

In [8]:
type_dict = nested_attr(type_sum, "funcs.d")
typs = trace_dispatch(type_dict); typs

(#3) [<Signature (a: int, b: int) -> int>,<Signature (a: float, b: int) -> float>,<Signature (a: float, b: float) -> float>]

In [9]:
#|export
@patch
def get_annotation(self:inspect.Signature,
    key:str,
):
    "Returns the annotation of `key` in `self`"
    return self.parameters[key].annotation

In [10]:
#|export
def get_dispatched(sigs:L):
    "Returns an ordered dictionary of key:types for each param in dispatch_signatures"
    var2types = {}
    for arg in sigs[0].parameters.keys():
        var2types[arg] = L(sigs).map(Self.get_annotation(arg))
    return var2types

In [11]:
#|export
def _dispatch_annotations(func):
    "Returns an ordered dictionary of key:types of all dispatch argument combinations in `func`"
    return get_dispatched(trace_dispatch(func))

In [12]:
_dispatch_annotations(type_sum)

{'a': (#3) [<class 'int'>,<class 'float'>,<class 'float'>],
 'b': (#3) [<class 'int'>,<class 'int'>,<class 'float'>]}

In [13]:
#|export
from fastcore.docments import *

In [14]:
type_dict = nested_attr(type_sum, "funcs.d")

In [15]:
#|export
def _dispatch_doc(func):
    "Returns the first docstring in a list of typedispatch'd functions"
    type_dict = nested_attr(func, "funcs.d")
    return first(L(type_dict.keys()).map(
            lambda x: [
                inspect.getdoc(o) 
                for o in type_dict[x].d.values()
            ]
        ).concat().filter(partial(is_not, None)))

In [16]:
_dispatch_doc(type_sum)

'Returns sum of `a` and `b`'

In [17]:
#|export
def docstring(sym):
    "Get docstring for `sym` for functions ad classes"
    if isinstance(sym, str): return sym
    if nested_attr(sym, "funcs.d"): return _dispatch_doc(sym)
    res = getdoc(sym)
    if not res and isclass(sym): res = getdoc(sym.__init__)
    return res or ""

In [18]:
docstring(type_sum)

'Returns sum of `a` and `b`'

In [19]:
#|export
def get_docmented_func(funcs):
    "Finds and returns the first func in `funcs` that has docments"
    dmented = None
    for func in L(f.d.values() for f in type_sum.funcs.d.values()).concat():
        dment = _docments(func, returns=False)
        if not all([k["docment"] == None for k in dment.values()]):
            return func
    return False

@patch
def __init__(self:DocmentTbl, obj, verbose=True, returns=True, dispatch=False):
    self.verbose = verbose
    self.returns = False if isdataclass(obj) else returns
    self.params = L(_signature(obj).parameters.keys())
    try:
        if dispatch:
            # Find the first set of docments and return them
            obj = get_docmented_func(dispatch)
            _dm = docments(obj, full=True, returns=returns)
            for k in _dm.keys(): _dm[k]["anno"] = inspect._empty
        else: _dm = docments(obj, full=True, returns=returns)
    except: _dm = {}
    if 'self' in _dm: del _dm['self']
    for d in _dm.values(): d['docment'] = ifnone(d['docment'], inspect._empty)
    self.dm = _dm

In [20]:
#|export
@patch
def __init__(self:ShowDocRenderer, sym, disp:bool=True):
    store_attr()
    self.nm = qual_name(sym)
    self.is_dispatch = nested_attr(sym, "funcs.d", False)
    self.isfunc = inspect.isfunction(sym)
    if self.is_dispatch:
        funcs = nested_attr(sym, "funcs.d")
        self.sigs = trace_dispatch(funcs)
        sym = first(L(f.d.values() for f in funcs.values()).concat())
        self.sig = _signature(sym)
        self.nm = qual_name(sym)
    else: self.sig = _signature(sym)
    self.docs = docstring(sym)
    self.dm = DocmentTbl(sym, dispatch=self.sigs if self.is_dispatch else False)

In [21]:
#|export
@patch
def _repr_markdown_(self:BasicMarkdownRenderer):
    doc = '---\n\n'
    if self.isfunc: doc += "#"
    doc += f'### {self.nm}\n\n'
    if self.is_dispatch:
        for sig in self.sigs: 
            sig = _wrap_sig(f"{self.nm} {_fmt_sig(sig)}")
            doc += f'{sig}\n'
    if self.docs: doc += f"\n\n{self.docs.splitlines()[0]}"
    if self.dm.has_docment: doc += f"\n\n{self.dm}"
    return doc

In [22]:
#|export
def show_doc(sym, disp=True, renderer=None):
    if renderer is None: renderer = get_config().get('renderer', BasicMarkdownRenderer)
    elif isinstance(renderer,str):
        p,m = renderer.rsplit('.', 1)
        renderer = getattr(import_module(p), m)
    return renderer(sym or show_doc, disp=disp)

In [23]:
show_doc(type_sum)

---

### type_sum

>      type_sum (a:int, b:int)
>      type_sum (a:float, b:int)
>      type_sum (a:float, b:float)


Returns sum of `a` and `b`

|    | **Details** |
| -- | ----------- |
| a | The first docment |
| b | The second docment |

This sets up a "proper" way to document typedispatch functions following **two key rules**:
1. Only a single function should ever be docmented, and these docments should describe what each value represents. The types live to show what the type is. So something like "A string", or "an int" is redundant.
2. All `typedispatch` functions should share the same docstring, as their core functionality should not change.

Note: this notebook cannot be exported because of a bug

```bash
Traceback (most recent call last):
  File "/opt/conda/bin/nbprocess_export", line 8, in <module>
    sys.exit(nbprocess_export())
  File "/opt/conda/lib/python3.9/site-packages/fastcore/script.py", line 113, in _f
    tfunc(**merge(args, args_from_prog(func, xtra)))
  File "/opt/conda/lib/python3.9/site-packages/nbprocess/doclinks.py", line 132, in nbprocess_export
    build_modidx()
  File "/opt/conda/lib/python3.9/site-packages/nbprocess/doclinks.py", line 102, in build_modidx
    if file.name[0]!='_': DocLinks(file, doc_func, _fn).build_index()
  File "/opt/conda/lib/python3.9/site-packages/nbprocess/doclinks.py", line 80, in build_index
    self.update_syms()
  File "/opt/conda/lib/python3.9/site-packages/nbprocess/doclinks.py", line 73, in update_syms
    exp += L(get_patch_name(o) for o in trees).filter()
  File "/opt/conda/lib/python3.9/site-packages/fastcore/foundation.py", line 97, in __call__
    return super().__call__(x, *args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/fastcore/foundation.py", line 105, in __init__
    items = listify(items, *rest, use_list=use_list, match=match)
  File "/opt/conda/lib/python3.9/site-packages/fastcore/basics.py", line 59, in listify
    elif is_iter(o): res = list(o)
  File "/opt/conda/lib/python3.9/site-packages/nbprocess/doclinks.py", line 73, in <genexpr>
    exp += L(get_patch_name(o) for o in trees).filter()
  File "/opt/conda/lib/python3.9/site-packages/nbprocess/doclinks.py", line 58, in get_patch_name
    if nm=='patch': pre = o.args.args[0].annotation.id
AttributeError: 'Attribute' object has no attribute 'id'
```